<a href="https://www.kaggle.com/code/dsptlp/clustering-kmeans-with-pca?scriptVersionId=162513879" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Clustering Using Kmeans
## Example 1 - Apple Quality
 - Description
 - Data Source 
## Example 2 - Mushrooms
 - Description
 - Data Source 

In [ ]:
!pip install polar >>install.log

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import LogNorm
import numpy as np 
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import warnings
import polar as pl

# Suppress all warnings
warnings.filterwarnings("ignore")

# EXAMPLE 1 - Apple Quality

In [ ]:
data_df = pd.read_csv('/kaggle/input/apple-quality/apple_quality.csv', delimiter=',') 
print(len(data_df))
data_df = data_df.drop('A_id', axis=1)
data_df = data_df[data_df['Acidity'] != 'Created_by_Nidula_Elgiriyewithana']
data_df.head(2)

In [ ]:
#data_corr_df = pl.analyze_correlation(data_df,'Quality')
#pl.get_heatmap(data_corr_df,'correlation_heat_map.png',1.1,14,'0.1f',0,100,5,5)

In [ ]:
sns.set_theme()
plt.figure()
sns.pairplot(data_df.replace([np.inf, -np.inf], np.nan), hue="Quality")
plt.show()

In [ ]:
scalar = StandardScaler()
df_std = scalar.fit_transform(data_df.drop('Quality', axis=1))

In [ ]:
# Convert these lists to numpy arrays.
X = np.array(df_std)
Y = np.array(data_df['Quality'])

# Split into train and test data.
train_data, train_labels = X[:], Y[:]

In [ ]:
vals = []

# Running for all possible values of k
k = range(1,(len(train_data[1])+1),1)
for comp_int in k:

    # Create PCA Model
    model = PCA(n_components = comp_int)
    model.fit(train_data)
    explained = np.sum(model.explained_variance_ratio_)
    print("Fraction of the total variance in the training data that is explained by %0.0f" %comp_int ,"components is = %0.2f %%" %(explained*100))
    vals.append(explained)

#Graph Results    
fig=plt.figure(figsize=(12,5))
plt.plot(k,vals,color='red')
plt.xlabel("Principal Components")
plt.ylabel("Fraction of the total variance")
plt.title("Fraction of the total variance in the training data VS Principal Components")
plt.show()

In [ ]:
 # Build the PCA Model
model_pca = PCA(n_components = (len(train_data[1])))
train_data_pca = model_pca.fit_transform(train_data)

# Quick examination of elbow method to find numbers of clusters to make.
print('Elbow Method to determine the number of clusters to be formed:')
Elbow_M = KElbowVisualizer(KMeans(n_init=10), k=7)
Elbow_M.fit(train_data_pca)
Elbow_M.show(outpath="elbow_plot.png")

optimal_k = Elbow_M.elbow_value_
print("Optimal number of clusters:", optimal_k)

In [ ]:
# Build the KMeans Model
model_kmeans = KMeans(n_clusters=optimal_k,init = 'k-means++',n_init='auto')
kmeans_clusters = model_kmeans.fit(train_data)
kmeans_clusters

data_df['kmean_clusters'] = kmeans_clusters.labels_
data_df.head(2)

In [ ]:
 # Build the PCA Model 2 DIMS
model_pca = PCA(n_components = 2)
train_data2DIM_pca = model_pca.fit_transform(train_data)
train_data2DIM_pca = pd.DataFrame(train_data2DIM_pca)
train_data2DIM_pca.columns = ['pca_2x','pca_2y']
train_data2DIM_pca.head(2)

In [ ]:
 # Build the PCA Model 3 DIMS
model_pca = PCA(n_components = 3)
train_data3DIM_pca = model_pca.fit_transform(train_data)
train_data3DIM_pca = pd.DataFrame(train_data3DIM_pca)
train_data3DIM_pca.columns = ['pca_3x','pca_3y','pca_3z']
train_data3DIM_pca.head(2)

In [ ]:
ff = pd.merge(data_df, train_data2DIM_pca, left_index=True, right_index=True)
ff = pd.merge(ff, train_data3DIM_pca, left_index=True, right_index=True)
ff

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D
from matplotlib import cm

#Plotting the clusters
fig = plt.figure(figsize=(10,8))
ax = plt.subplot(111, projection='3d')
ax.scatter(ff["pca_3x"], ff["pca_3y"], ff["pca_3z"], s=20, c=ff["kmean_clusters"], marker='o',cmap='viridis' )
ax.set_title("Clusters represented Via PCA in 3D")

# Creating a legend based on unique cluster values
legend_labels = ff["kmean_clusters"].unique()
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=cm.viridis(label), markersize=10) for label in legend_labels]
ax.legend(legend_elements, legend_labels, title='Cluster')

plt.show()

In [ ]:
# Assuming you have a dataframe 'ff' and it has a column 'Quality'
# Convert 'Quality' to categorical for color mapping
ff['Quality'] = ff['Quality'].astype('category')
categories = ff['Quality'].cat.categories

# Plotting the clusters
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Assigning unique colors to each category
colors = plt.cm.viridis(range(len(categories)))

# Scatter plot with categorical colors
scatter = ax.scatter(
    ff["pca_3x"], ff["pca_3y"], ff["pca_3z"],
    s=20, c=ff['Quality'].cat.codes, cmap='viridis', marker='o'
)

ax.set_title("Clusters represented Via PCA in 3D")

# Adding colorbar for better visualization of category values
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(len(categories)))
cbar.set_ticklabels(categories)
cbar.set_label('Quality')

plt.show()


In [ ]:
ff.loc[data_df["Quality"] == "good", "QualityBin"] = 1
ff.loc[data_df["Quality"] == "bad", "QualityBin"] = 0
ff

In [ ]:
columns = [ "Size","Weight","Sweetness","Crunchiness","Juiciness","Ripeness"]

# Number of clusters or unique values in the "kmean_clusters" variable
num_clusters = len(ff["kmean_clusters"].unique())

# Define a custom color palette with distinct colors for each cluster
custom_palette = sns.color_palette("husl", n_colors=num_clusters)

# Assuming ff is your DataFrame
for i in columns:
    plt.figure()
    sns.jointplot(x=ff[i], y=ff["QualityBin"], hue=ff["kmean_clusters"], kind="kde", palette=custom_palette)
    plt.show()

In [ ]:
columns = [ "Size","Weight","Sweetness","Crunchiness","Juiciness","Ripeness"]

for i in columns:
    plt.figure()
    sns.jointplot(x=ff[i], y=ff["QualityBin"], hue =ff["kmean_clusters"])
    plt.show()

# EXAMPLE 2 - Mushrooms

In [ ]:
data_df = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv', delimiter=',') 
print(len(data_df))
data_df.head(5)

In [ ]:
data_df.loc[data_df["class"] == "p", "class"] = 1
data_df.loc[data_df["class"] == "e", "class"] = 0
data_df['class'] = data_df['class'].astype(int)
data_df.head(2)

In [ ]:
data_corr_df = pl.analyze_association(data_df,'class',verbose=0)
pl.get_heatmap(data_corr_df,'association_heat_map.png',1.1,12,'0.1f',0,100,10,10)

In [ ]:
# Perform one-hot encoding using get_dummies
df_encoded = pd.get_dummies(data_df)
df_encoded.head(2)

In [ ]:
print(df_encoded.columns)

In [ ]:
# Convert these lists to numpy arrays.
X = np.array(df_encoded.drop('class', axis=1))
Y = np.array(df_encoded['class'])

# Split into train and test data.
train_data, train_labels = X[:7000], Y[:7000]
test_data, test_labels = X[7000:], Y[7000:]

In [ ]:
vals = []

# Running for all possible values of k
k = range(1,(len(train_data[1])+1),1)
for comp_int in k:

    # Create PCA Model
    model = PCA(n_components = comp_int)
    model.fit(train_data)
    explained = np.sum(model.explained_variance_ratio_)
    print("Fraction of the total variance in the training data that is explained by %0.0f" %comp_int ,"components is = %0.2f %%" %(explained*100))
    vals.append(explained)

#Graph Results    
fig=plt.figure(figsize=(12,5))
plt.plot(k,vals,color='red')
plt.xlabel("Principal Components")
plt.ylabel("Fraction of the total variance")
plt.title("Fraction of the total variance in the training data VS Principal Components")
plt.show()

In [ ]:
 # Build the PCA Model
model_pca = PCA(n_components = 8)
train_data_2dims = model_pca.fit_transform(train_data)

In [ ]:
scores_pca = model_pca.transform(train_data)
scores_pcs_df = pd.DataFrame(scores_pca)
scores_pcs_df

In [ ]:
# Quick examination of elbow method to find numbers of clusters to make.
print('Elbow Method to determine the number of clusters to be formed:')
Elbow_M = KElbowVisualizer(KMeans(n_init=10), k=10)
Elbow_M.fit(train_data_2dims)
Elbow_M.show()

In [ ]:
# Build PCA Model with 2 PCA Components
model = PCA(n_components = 2)
train_data_2dims = model.fit_transform(train_data)

# Graph Results
fig=plt.figure(figsize=(10,10))
plt.scatter(train_data_2dims[:,0][train_labels==1],train_data_2dims[:,1][train_labels==1],marker="o",color='red', label="poisonous",edgecolor='black',s=35,alpha=0.7)
plt.scatter(train_data_2dims[:,0][train_labels==0],train_data_2dims[:,1][train_labels==0],marker="o",color='green', label="non-poisonous",edgecolor='black',s=35,alpha=0.7)    
plt.xlabel("First PCA Dimension")
plt.ylabel("Second PCA Dimension")
plt.legend()
plt.title("2 Dimension PCA Visualization")
plt.show()

In [ ]:
# Build the PCA Model
model_pca = PCA(n_components = 2)
train_data_2dims = model_pca.fit_transform(train_data)

# Build the KMeans Model
model_kmeans = KMeans(n_init=10,n_clusters=6)
kmeans_clusters = model_kmeans.fit(train_data_2dims)

# Create The Graph
fig=plt.figure(figsize=(10,10))
plt.xlim([-3.3,3.7])
plt.ylim([-3.3,3.7])
plt.scatter(train_data_2dims[:,0][train_labels==1],train_data_2dims[:,1][train_labels==1],marker="o",color='red', edgecolor='silver',label="poisonous")
plt.scatter(train_data_2dims[:,0][train_labels==0],train_data_2dims[:,1][train_labels==0],marker="o",color='green', edgecolor='silver',label="non-poisonous")
plt.scatter(kmeans_clusters.cluster_centers_[:,0],kmeans_clusters.cluster_centers_[:,1],marker="o",color='black', label="cluster centrioid",s=110,alpha=0.8)

#Get All Cluster Centers
for idx,centerxy in enumerate(kmeans_clusters.cluster_centers_):

    #Set Radious to zero
    circle_radius = 0

    # Determine the distance between the center and find the largest radius
    for xy_point in train_data_2dims[kmeans_clusters.labels_ == idx]:
        new_circle_rad = np.linalg.norm(centerxy-xy_point)

        #if new radius is bigger set it as the one we want
        if new_circle_rad > circle_radius:
            circle_radius = new_circle_rad

    # Draw the circle
    circle = plt.Circle(centerxy,circle_radius,alpha=0.1,edgecolor='black')        
    plt.gca().add_artist(circle)    

plt.xlabel("First PCA Dimension")
plt.ylabel("Second PCA Dimension")
plt.legend()
plt.title("2 Dimension PCA Visualization")
plt.show()

In [ ]:
# Declare the required variable combinations
cov_matrix_type_list = ['spherical', 'diag', 'tied', 'full']
components_list = [1,2,3,4]

# Build PCA Model
model_pca = PCA(n_components = 2)
train_data_2dims = model_pca.fit_transform(train_data)

# Only for Positive Cases
pos_ex = train_data_2dims[train_labels==1]

# Declare fig subplot
fig, ax = plt.subplots(figsize=(25,65) , sharex=True,sharey=True)

# Loop for the required combinations
for idx, cov_matrix_type in enumerate(cov_matrix_type_list):
    for idx2, components in enumerate(components_list):

        #Create GMM Model
        model_GMM = GaussianMixture(n_components=components,covariance_type=cov_matrix_type,random_state=12345)
        model_GMM.fit(pos_ex)

        #Make the contour 
        plt.subplot(10,4,1+(4*idx2+idx)) 

        # This portion of code is from the provided example https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html#example-mixture-plot-gmm-pdf-py  
        x = np.linspace(-3.3,3.7)
        y = np.linspace(-3.3,3.7)            
        X, Y = np.meshgrid(x, y)
        XX = np.array([X.ravel(), Y.ravel()]).T
        Z = -model_GMM.score_samples(XX)
        Z = Z.reshape(X.shape)            
        CS = plt.contour(X, Y, Z)            
        CB = plt.colorbar(CS, shrink=0.8)

        #Plot the graph
        plt.scatter(train_data_2dims[:,0][train_labels==1],train_data_2dims[:,1][train_labels==1],marker="o",color='red')
        plt.xlabel("First PCA Dimension")
        plt.ylabel("Second PCA Dimension")
        plt.title("Component = %0.0f" % components  + ", Covariance Matrix Type = " + str(cov_matrix_type))
        plt.subplots_adjust(hspace=0.3,wspace=0.08)

# Save the entire subplot to an image file
plt.savefig("cov_plot.png")        
        
#show the entire subplot
plt.show()

In [ ]:
# Build PCA Model
model_pca = PCA(n_components = 2)
train_data_2dims = model_pca.fit_transform(train_data)
test_data_2dims = model_pca.transform(test_data)

# Split data into pos and neg
pos_ex = train_data_2dims[train_labels==1]
neg_ex = train_data_2dims[train_labels==0]

#Create GMM Model for positive
model_GMM_pos = GaussianMixture(n_components=4,covariance_type='full',random_state=12345)
model_GMM_pos.fit(pos_ex)

#Create GMM Model for negative
model_GMM_neg = GaussianMixture(n_components=4,covariance_type='full',random_state=12345)
model_GMM_neg.fit(neg_ex)

#accuracy calculation
higher_prob_label_case = (model_GMM_pos.score_samples(test_data_2dims) > model_GMM_neg.score_samples(test_data_2dims)).astype(int)
correct_predictions = np.where(higher_prob_label_case == np.array(test_labels),1,0)

print("The accuracy of the predictions on the test data is = %0.02f %%" % ((correct_predictions.sum()/test_labels.size)*100))

In [ ]:
cov_matrix_type_list = ['spherical', 'diag', 'tied', 'full']
pca_components_list = range(1,100,1)
gmm_components_list = range(1,100,1)
result = []
highest_accuracy = 0

print("Running experiments:")

# For all the possible combinations of pca/gmm and cov_matrix_type
for cov in cov_matrix_type_list:
    for pca in pca_components_list:
        for gmm in gmm_components_list:

            # Based on the provided parameter count definition
            if cov == 'tied':
                param_count = 2 * ((pca * gmm) + (pca*(pca+1)/2 ))
            elif cov == 'spherical':
                param_count = 2 * (gmm * pca + gmm)
            elif cov == 'diag':
                param_count = 4 * gmm * pca
            elif cov == 'full':
                param_count = 2 * ((pca * gmm) + (pca*(pca+1)/2 * gmm ))

            # Only calculate if the parameter count is <= 50
            if param_count <= 50:

                # Build the PCA Model
                model_pca = PCA(n_components = pca)
                train_data_trans = model_pca.fit_transform(train_data)
                test_data_trans = model_pca.transform(test_data)

                # Create positive and negative cases
                pos_ex = train_data_trans[train_labels==1]
                neg_ex = train_data_trans[train_labels==0]

                # Build the GMM model for positive 
                model_GMM_pos = GaussianMixture(n_components=gmm, covariance_type=cov, random_state=12345) 
                model_GMM_pos.fit(pos_ex)

                # Build the GMM model for negative
                model_GMM_neg = GaussianMixture(n_components=gmm, covariance_type=cov, random_state=12345) 
                model_GMM_neg.fit(neg_ex)

                #accuracy calculation, taking the better of pos/neg
                higher_prob_label_case = (model_GMM_pos.score_samples(test_data_trans) > model_GMM_neg.score_samples(test_data_trans)).astype(int)
                correct_predictions = np.where(higher_prob_label_case == np.array(test_labels),1,0)
                accuracy = ((correct_predictions.sum()/test_labels.size)*100)

                # Records the highest accuracy configuration
                if accuracy > highest_accuracy:
                    result.append({'cov':cov,'pca':pca,'gmm':gmm,'param_count':param_count,'accuracy':accuracy})
                    highest_accuracy = accuracy
                print("Covariance = " + cov + " | PCA Components = %0.f" % pca + " | GMM Components = %0.f " % gmm + " | Parameters Count = %0.0f" % param_count + " | Accuracy = %0.2f %%" %accuracy)

print ("=====================================================================================================================")
print("Gaussian mixture model that results in the best accuracy with no more than 50 parameters is:")    
cov = result[(len(result)-1)]['cov']
pca = result[(len(result)-1)]['pca']
gmm = result[(len(result)-1)]['gmm']
param_count = result[(len(result)-1)]['param_count']
accuracy = result[(len(result)-1)]['accuracy']
print("Covariance = " + cov + " | PCA Components = %0.f" % pca + " | GMM Components = %0.f " % gmm + " | Parameters Count = %0.0f" % param_count + " | Accuracy = %0.2f %%" %accuracy)